# Datasettype Loading

## Imports

In [ ]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load
import pandas as pd
from pathlib import Path

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Datasettype

In [ ]:
datasettpypes_table = qrs.get_data('datasettypes')

datasettpypes_table

In [ ]:
datasettype = 5

## Dataset

In [16]:
dataset_paths = [str(Path(path)) for path in [
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_4\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_5\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_7\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_8\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_10\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_11\MonoElement\patch_vs_volfrac_9.csv',
    r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_12\MonoElement\patch_vs_volfrac_9.csv'
]]

patch_size = '9'

registration_id = [75]

targets = ['Volfrac']

reconstruction_shape = (-1,-1)

for dataset_path in dataset_paths:
    dataset = pd.read_csv(dataset_path)
    rows = len(dataset)
    load.load_dataset(conn=conn,datasettype_id=datasettype,file_path=dataset_path,rows=rows,patch_size=patch_size,targets=targets,reconstruction_shape=reconstruction_shape,registration_ids=registration_id)


Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_4\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 293
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_5\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 294
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_7\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 295
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_8\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 296
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_10\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 297
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_11\MonoElement\patch_vs_volfrac_9.csv' loaded with ID: 298
Dataset from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI

## Make sure insert was correct

In [ ]:
data = qrs.get_data('datasettypes')

data

In [ ]:
conn.close()